In [37]:
import os
import pandas as pd

In [38]:
text = """
Gli abitanti di Little Hangleton la chiamavano ancora Casa Riddle, anche se erano passati tanti anni da quando i Riddle ci abitavano. 
Si trovava sulla collina che dominava il villaggio: alcune delle finestre erano inchiodate, al tetto mancavano delle tegole e 
l'edera cresceva incolta sulla facciata. Un tempo Casa Riddle era stata una dimora elegante, certo l'edificio più vasto e grandioso 
nel raggio di chilometri, ma ora era umida, desolata e disabitata. Gli hangletoniani convenivano tutti che la vecchia casa era 
sinistra. Mezzo secolo prima, qualcosa di strano e terribile era successo là dentro, qualcosa di cui gli abitanti più anziani del 
villaggio amavano ancora discutere quando erano a corto di pettegolezzi. La storia era stata ripetuta così tante volte, e vi erano 
stati aggiunti così tanti fronzoli che nessuno era più certo di quale fosse la verità. Ogni versione del racconto, comunque, 
cominciava allo stesso modo: cinquant'anni prima, all'alba di una bella giornata d'estate, quando Casa Riddle era ancora ben tenuta e 
imponente, una cameriera era entrata in salotto e aveva trovato morti tutti e tre i Riddle. La cameriera era corsa urlando giù per la 
collina fino al villaggio e aveva radunato tutte le persone che poteva. Sono là stesi con gli occhi spalancati! Freddi come il 
ghiaccio! Ancora vestiti per la cena! Fu chiamata la polizia e tutta quanta Little Hangleton si crogiolò in una curiosità atterrita e 
in una malcelata eccitazione. Nessuno si sforzò di fingersi addolorato per i Riddle, che erano stati assolutamente impopolari. Gli 
anziani signori Riddle, marito e moglie, erano ricchi, snob e sgarbati, e loro figlio Tom, ormai adulto, era anche peggio. Tutto 
quello che importava agli abitanti era l'identità dell'assassino: chiaramente, tre persone in apparenza sane non morivano di colpo per 
cause naturali nella stessa notte. L'Impiccato, il pub locale, fece affari d'oro quella sera: il villaggio al completo accorse per 
discutere gli omicidi. E la ricompensa per quell'uscita serale arrivò quando la cuoca dei Riddle fece un ingresso teatrale e annunciò 
al pub improvvisamente silenzioso che un uomo chiamato Frank Bryce era stato appena arrestato.
"""

In [39]:
def get_all_csv_files_rec(dir):
    """Ritorna una lista di tutti i file .csv nella directory e nelle sottodirectory."""
    csv_files = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files
all_csv_files = get_all_csv_files_rec('dataframes')

In [40]:
all_df = pd.DataFrame()
for path in all_csv_files:
    assert(os.path.exists(path))
    # print(path)
    df = pd.read_csv(path)
    all_df = pd.concat([all_df, df], ignore_index=True)
len(all_df), all_df.columns

(4414,
 Index(['term_italian', 'is_base', 'base_lemma_italian', 'base_lemma_english',
        'part_of_speech', 'translation_english', 'topics', 'subtype',
        'person_number', 'gender', 'plurality', 'mood', 'tense',
        'is_comparative', 'is_superlative', 'is_compound', 'article_type',
        'article_italian', 'added_particle_italian', 'preposition', 'notes',
        'example_sentence_italian', 'example_sentence_english'],
       dtype='object'))

In [41]:
all_df.drop_duplicates(['term_italian', 'base_lemma_italian', 'base_lemma_english', 'part_of_speech'], inplace=True)
len(all_df)

4272

In [42]:
all_df['base'] = all_df.apply(lambda row: f"{row['base_lemma_italian']} - {row['base_lemma_english']}", axis=1)
all_df['base'].value_counts()

base
dovere - obligation; must/have to    41
ricco - rich                         38
impiccato - hanged                   38
cuoco - cook                         34
essere - to be                       34
                                     ..
là - there                            1
o - or                                1
anche - also                          1
ma - but                              1
tra - between/among                   1
Name: count, Length: 327, dtype: int64

In [43]:
all_df['base'].value_counts()[-10:]
for base in all_df['base'].value_counts()[-10:].index:
    # print base and part_of_speech
    base_df = all_df[all_df['base'] == base]
    print(f"Base: {base}, Part of speech: {base_df['part_of_speech'].iloc[0]}")

Base: noi - we, Part of speech: pronoun
Base: ci - us/there, Part of speech: pronoun
Base: vi - you (plural)/there, Part of speech: pronoun
Base: ormai - by now, Part of speech: adverb
Base: sé - oneself, Part of speech: pronoun
Base: là - there, Part of speech: adverb
Base: o - or, Part of speech: conjunction
Base: anche - also, Part of speech: adverb
Base: ma - but, Part of speech: conjunction
Base: tra - between/among, Part of speech: preposition


In [44]:
all_df[all_df['base'] == 'ognuno - everyone (feminine)']

,term_italian,is_base,base_lemma_italian,base_lemma_english,part_of_speech,translation_english,topics,subtype,person_number,gender,...,is_superlative,is_compound,article_type,article_italian,added_particle_italian,preposition,notes,example_sentence_italian,example_sentence_english,base


In [45]:
ignore_words = [
    'bryce',
    "cinquant'anni",
    'frank',
    'hangleton',
    'little',
    'riddle',
    'tom',
]
harry_potter_words = set(text.lower().replace('\n', ' ').replace(':', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace('?', ' ').split(' '))
# ignore empty strings
harry_potter_words.discard('')
# ignore numbers
harry_potter_words = {w for w in harry_potter_words if not w.isdigit()}
harry_potter_words = harry_potter_words - set(ignore_words)
len(harry_potter_words)

207

In [46]:
IMPORTANT_COLS = ['term_italian', 'part_of_speech', 'translation_english', 'base_lemma_italian', 'base_lemma_english']

In [47]:
all_df[all_df['term_italian'].isin(harry_potter_words)][IMPORTANT_COLS]

,term_italian,part_of_speech,translation_english,base_lemma_italian,base_lemma_english
1,appena,conjunction,as soon as,appena,just/barely; as soon as
3,e,conjunction,and,e,and
7,più,conjunction,plus,più,more/plus
10,come,conjunction,as/like,come,as/like/how
13,comunque,conjunction,however/no matter how,comunque,however
...,...,...,...,...,...
4401,dei,preposition,of the,di,of
4403,delle,preposition,of the,di,of
4404,in,preposition,in/into,in,in/into
4405,nel,preposition,in the,in,in/into


In [ ]:
missing_words = harry_potter_words - set(all_df['term_italian'])
print(len(missing_words))
missing_words

2


{'crogiolò', 'hangletoniani'}

In [ ]:
# The residents of Little Hangleton still called it Riddle House, even though it had been many years since the Riddles had lived there.
sentence_1 = "Gli abitanti di Little Hangleton la chiamavano ancora Casa Riddle, anche se erano passati tanti anni da quando i Riddle ci abitavano."

# It stood on the hill overlooking the village: some of the windows were nailed shut, tiles were missing from the roof, and ivy was growing wild on the facade.
sentence_2 = "Si trovava sulla collina che dominava il villaggio: alcune delle finestre erano inchiodate, al tetto mancavano delle tegole e l'edera cresceva incolta sulla facciata."

# Riddle House had once been an elegant mansion, certainly the largest and grandest building for miles around, but now it was damp, desolate, and uninhabited.
sentence_3 = "Un tempo Casa Riddle era stata una dimora elegante, certo l'edificio più vasto e grandioso nel raggio di chilometri, ma ora era umida, desolata e disabitata."

# Hangletonians all agreed that the old house was sinister.
sentence_4 = "Gli hangletoniani convenivano tutti che la vecchia casa era sinistra."

sentence = sentence_2

In [63]:
sentence_italian_words = list(sentence.lower().replace('\n', ' ').replace(':', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace('?', ' ').split(' '))
sentence_italian_words = [w for w in sentence_italian_words if w != '']
sentence_italian_words

['si',
 'trovava',
 'sulla',
 'collina',
 'che',
 'dominava',
 'il',
 'villaggio',
 'alcune',
 'delle',
 'finestre',
 'erano',
 'inchiodate',
 'al',
 'tetto',
 'mancavano',
 'delle',
 'tegole',
 'e',
 "l'edera",
 'cresceva',
 'incolta',
 'sulla',
 'facciata']

In [64]:
all_df.columns

Index(['term_italian', 'is_base', 'base_lemma_italian', 'base_lemma_english',
       'part_of_speech', 'translation_english', 'topics', 'subtype',
       'person_number', 'gender', 'plurality', 'mood', 'tense',
       'is_comparative', 'is_superlative', 'is_compound', 'article_type',
       'article_italian', 'added_particle_italian', 'preposition', 'notes',
       'example_sentence_italian', 'example_sentence_english', 'base'],
      dtype='object')

In [65]:
from fuzzy_match import find_translation

In [66]:
all_df[all_df['term_italian'] == 'gli']

,term_italian,is_base,base_lemma_italian,base_lemma_english,part_of_speech,translation_english,topics,subtype,person_number,gender,...,is_superlative,is_compound,article_type,article_italian,added_particle_italian,preposition,notes,example_sentence_italian,example_sentence_english,base
4053,gli,True,gli,to him/to them,pronoun,to him,"core,pronoun",indirect_object,3s,masculine,...,False,False,NaN,NaN,NaN,NaN,clitic indirect object pronoun; placed before ...,Io gli parlo adesso.,I am speaking to him now.,gli - to him/to them
4186,gli,False,il,the,article,"the (masculine plural, before vowels, s+conson...","core,grammar",definite,NaN,masculine,...,False,False,NaN,NaN,NaN,NaN,"before vowels, s+consonant, z, gn, ps, x, y (m...",Vedo gli studenti.,I see the students.,il - the


In [67]:
# find_translation(all_df, 'crogiolò', min_similarity=0.8)
# find_translation(all_df, 'little', min_similarity=0.9)

In [68]:
def get_trans(word_italian):
    ret = find_translation(all_df, word_italian, min_similarity=0.8)
    if ret is None:
        return word_italian, None, None
    it, en = ret
    return word_italian, it, en
sentence_english_trans = [
    get_trans(word) for word in sentence_italian_words
]
new_sentence = ''
for original_word, match_it, match_en in sentence_english_trans:
    new_sentence += f"{original_word} [{match_en}] "
new_sentence

"si [['yes', 'himself/herself/itself/themselves/yourself (formal)']] trovava [he was finding/she was finding/you (formal) were finding] sulla [on the] collina [hill] che [['that', 'who/that/which']] dominava [he was dominating/she was dominating/you (formal) were dominating] il [the (masculine singular)] villaggio [village] alcune [some] delle [['some', 'of the']] finestre [windows] erano [they were] inchiodate [['nailed/slammed on the brakes', 'nailed (feminine plural)']] al [to the (masculine singular)] tetto [roof] mancavano [they were missing] delle [['some', 'of the']] tegole [roof tiles] e [['and', 'he is/she is/you (formal) are']] l'edera [the ivy] cresceva [he was growing/she was growing/you (formal) were growing] incolta [uncultivated (feminine)] sulla [on the] facciata [facade] "

In [ ]:
sentence_italian_words

['gli',
 'abitanti',
 'di',
 'little',
 'hangleton',
 'la',
 'chiamavano',
 'ancora',
 'casa',
 'riddle',
 'anche',
 'se',
 'erano',
 'passati',
 'tanti',
 'anni',
 'da',
 'quando',
 'i',
 'riddle',
 'ci',
 'abitavano']

In [36]:
i = 7
it_word = sentence_1_italian_words[i]
print(f"{it_word} - {find_translation(all_df, it_word, min_similarity=0.8)}")

ancora - ('ancora', ['anchor', 'he anchors/she anchors/you (formal) anchor', 'still/again/yet'])
